In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
def get_musics_for_tv_series(driver_path, links):
    """
    links: array for tunefind links for each tv series
    """
    c = webdriver.ChromeOptions()
    c.add_argument("--incognito")

    driver = webdriver.Chrome(driver_path,options=c)
    driver.maximize_window()
    
    # loop through each link
    for link in links:
        driver.get(link)
        
        series_name = "" 
        for name in link.split("show/")[1].split("-"):
            series_name = series_name + name.capitalize() + " "
        series_name = series_name.strip()
        print(series_name)
        
        seasons = driver.find_elements(by=By.CLASS_NAME, value = 'MainList_item__bR_g9') # get the seasons
        
        not_season_counter = 0 # there are some adds between the seasons div
        # loop through each season
        for idx, season in enumerate(seasons): 

            time.sleep(2)

            df = pd.DataFrame(columns = ['series_name','track_id'])
            
            # try to get the season link
            try:
                season_link = season.find_element(by=By.TAG_NAME, value = 'a')
                season_href = season_link.get_attribute('href')
            except: # if you cant get season link, this means it is an addvertisement
                not_season_counter += 1
                continue

            all_seasons_tab = driver.window_handles[0] # get the tab that contains all seasons

            driver.execute_script("window.open('');") # open a new empty tab
            time.sleep(0.5)
            season_tab = driver.window_handles[1] # get that empty tab

            driver.switch_to.window(season_tab) # switch to empty tab
            driver.get(season_href) # and go to season tab that contains each episode in the tab

            episodes = driver.find_elements(by=By.CLASS_NAME, value = "MainList_item__bR_g9") # get all episodes
            
            # loop through each 
            for episode in episodes:
                try:
                    episode_header = episode.find_element(by=By.CLASS_NAME, value = "EpisodeListItem_title__PkSzj") # get the episode header

                    episode_href = episode_header.find_element(by = By.TAG_NAME, value = "a").get_attribute('href') # get the episode link

                    driver.execute_script("window.open('');") # open an empty tab
                    time.sleep(0.5)
                    episode_tab = driver.window_handles[2] # get that empty tab

                    driver.switch_to.window(episode_tab) # switch to empty tab
                    driver.get(episode_href) # and go the episode tab that contains all the songs in that epsiode
                    
                    # try to get the song list
                    try:
                        song_list_container = driver.find_element(by=By.CLASS_NAME, value = "SongList_container__GU1aF")
                    except: # that means there is no song in that episode
                        print("There is no song in this episode.")
                        
                        driver.close() # close that episode tab
                        driver.switch_to.window(season_tab) # switch to season tab

                        continue
                    
                    # if there is song get the song list
                    spotify_buttons = song_list_container.find_elements(by=By.CLASS_NAME, value = "StoreLinks_spotify__QjCyP")

                    num_of_buttons = len(spotify_buttons) # get the number of songs
                    print("number of songs in this episode :",num_of_buttons)

                    button_idx = 0
                    # loop throguh each spotify button
                    while button_idx < num_of_buttons:
                        time.sleep(1)

                        driver.execute_script("arguments[0].click();", spotify_buttons[button_idx]) # click to spotify button
                        
                        # try to get the spotify tab
                        try:
                            spotify_song_tab = driver.window_handles[3]
                        except Exception as e: # if you cant get the spotify tab, song is not in spotify
                            print('song is not on spotify')
                            button_idx += 1
                            continue # continue with the next song

                        driver.switch_to.window(spotify_song_tab) # switch to spotify tab
                        
                        # try to get the track id from the url
                        try:
                            song_track_id = driver.current_url.split('track/')[1] #get the track id
                        # sometimes selenium throws an unknown error as "unknown error: cannot determine loading status"
                        except Exception as e:
                            print('can not add the song, try it again')
                            driver.close() # close the spotify page
                            time.sleep(1)
                            driver.switch_to.window(episode_tab) # switch to episode tab again, dont increment the button counter becuase we will try to add this song again
                            continue


                        df = df.append({'series_name' : series_name, 'track_id' : song_track_id},ignore_index = True) # add the track id and corresponding tv series name to dataframe
                        print(song_track_id,"is added.")

                        button_idx += 1

                        driver.close() # close the song tab
                        #time.sleep(1) 
                        driver.switch_to.window(episode_tab) # swtich to episode tab


                    driver.close() # close the episode tab
                    #time.sleep(1)
                    driver.switch_to.window(season_tab) # switch to season tab

                except: # this means it is not episode continue with the next div
                    print("it is not an episode.")
                    continue

            file_name = f"data_2/{series_name}_{idx + 1 - not_season_counter}.csv"
            df.to_csv(file_name) # save the dataframe for the season

            driver.close() # close the season tab
            driver.switch_to.window(all_seasons_tab) # switch to all season tab
    
    driver.close()

In [4]:
links = ['https://www.tunefind.com/show/sons-of-anarchy','https://www.tunefind.com/show/silicon-valley','https://www.tunefind.com/show/vikings']
get_musics_for_tv_series("C:/Users/Omer/Documents/music_tv_series/chromedriver.exe",links)

C:\Users\Omer\AppData\Local\Temp\ipykernel_18680\2248095568.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path,options=c)


Vikings
number of songs in this episode : 11


C:\Users\Omer\AppData\Local\Temp\ipykernel_18680\2248095568.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'series_name' : series_name, 'track_id' : song_track_id},ignore_index = True) # add the track id and corresponding tv series name to dataframe


1we2UifyzGe2tgvTVFTwD4 is added.


C:\Users\Omer\AppData\Local\Temp\ipykernel_18680\2248095568.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'series_name' : series_name, 'track_id' : song_track_id},ignore_index = True) # add the track id and corresponding tv series name to dataframe


...

2Pd9RW3hoQzTT2pFcOE5sM is added.


C:\Users\Omer\AppData\Local\Temp\ipykernel_18680\2248095568.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'series_name' : series_name, 'track_id' : song_track_id},ignore_index = True) # add the track id and corresponding tv series name to dataframe


26jgMDo8CR33rxt9CEMymT is added.


C:\Users\Omer\AppData\Local\Temp\ipykernel_18680\2248095568.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'series_name' : series_name, 'track_id' : song_track_id},ignore_index = True) # add the track id and corresponding tv series name to dataframe


0yEAaSZGR9TlcPkFqpZzAf is added.
There is no song in this episode.
